# Blink Detection using MediaPipe and OpenCV


In this notebook, we will detect human eye blinks in real-time using a metric called the Eye Aspect Ratio (EAR). A blink is identified when the EAR falls below a specified threshold for a certain number of consecutive frames. 

We will explore how to fine-tune the EAR threshold and the frame count parameter to optimize blink detection accuracy. Additionally, we will demonstrate the impact of these parameters on the system's performance and plot the EAR in real-time while counting blinks. This visualization will provide valuable insights into the blink detection process.

## Content

1. [Imports](#1-importing-necessary-libraries)
2. [Eye Landmark Detection](#2-eye-landmark-detection)
3. [Drawing specific EAR Landmarks](#3-drawing-the-specific-ear-landmarks)
4. [EAR Analysis](#4-analyzing-the-eye-aspect-ratioear)
    - [Blink Detection with Multiple Thresholds](#41-blink-detection-with-multiple-ear-thresholds)
    - [Blink Detection with varying Consecutive Frames](#42-blink-detection-with-varying-consecutive-frames)
5. [Blink Detection and Counting](#5-blink-detection-and-counting)
6. [Real-Time Blink Detection with EAR plotting](#6-real-time-eye-blink-detection-and-visualization-with-ear-plotting)
7. [Applications](#7-applications)
8. [References](#8-references)

## 1. Importing necessary libraries

Let's start off by importing some libraries that will be required during the blink detection process.

> - **MediaPipe** : face mesh detection and landmark extraction
> - **OpenCv**: image and video processing
> - **Matplotlib** : for plotting and visualizing data
> - **FaceMeshModule** : for generating face mesh and landmarks
> - **Plotly**: for interactive and dynamic plotting

In [ ]:
import mediapipe as mp
import cv2 as cv
import numpy as np
import os
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from FaceMeshModule import FaceMeshGenerator
from utils import DrawingUtils
import plotly.graph_objects as go
from plotly.subplots import make_subplots


## 2. Eye Landmark Detection
To enable blink detection, the first crucial step is to identify the landmarks of the eyes in the input image. This class, `DetectEyeLandmarks`, leverages MediaPipe's FaceMesh to extract facial landmarks and specifically highlights the eye regions.

Key Features:
- Detects eye landmarks for both right and left eyes.
- Visualizes landmarks by overlaying colored circles on the input image.
- Saves the processed image if desired.

Parameters:
- **image_path**: Path to the input image.
- **save_img**: Set to `True` if you want to save the annotated image.
- **filename**: Name of the output file for saving.

Below is the implementation of the `DetectEyeLandmarks` class.


In [ ]:
class DetectEyeLandmarks:
    # Eye landmark indices
    RIGHT_EYE = [33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161, 246]
    LEFT_EYE = [362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385, 384, 398]
    
    # Colors
    GREEN_COLOR = (0, 255, 0)  # Right eye color
    RED_COLOR = (0, 0, 255)   # Left eye color
    
    def __init__(self, image_path, save_img=False, filename=None):
        """
        Initialize the eye landmark detector.
        
        Args:
            image_path (str): Path to input image
            save_img (bool): Whether to save the processed image
            filename (str): Output filename if saving image
        """
        self.image_path = image_path
        self.filename = filename
        self.save_img = save_img
        self.generator = FaceMeshGenerator()
        
        # Setup save directory if needed
        if self.save_img:
            self._setup_save_directory()
            
        # Process image
        self.image = self._load_image()
        if self.image is not None:
            self._process_image()
    
    def _setup_save_directory(self):
        """Create directory for saving images if it doesn't exist."""
        if not self.filename:
            raise ValueError("Filename must be provided when save_img is True")
            
        save_dir = "DATA/IMAGES/BLINK_DETECTION"
        os.makedirs(save_dir, exist_ok=True)
        self.filename = os.path.join(save_dir, self.filename)

    def _load_image(self):
        """Load and validate input image."""
        try:
            img = cv.imread(self.image_path)
            if img is None:
                raise FileNotFoundError(f"Could not load image from {self.image_path}")
            return img
        except Exception as e:
            print(f"Error loading image: {str(e)}")
            return None

    def _process_image(self):
        """Process image to detect and draw landmarks."""
        try:
            # Detect face landmarks
            self.image, face_landmarks = self.generator.create_face_mesh(self.image, draw=False)
            
            if not face_landmarks:
                raise ValueError("No face detected in the image")
                
            # Draw landmarks for both eyes
            self._draw_eye_landmarks(face_landmarks, self.RIGHT_EYE, self.GREEN_COLOR)
            self._draw_eye_landmarks(face_landmarks, self.LEFT_EYE, self.RED_COLOR)
            
            # Display and save results
            self.plot_eye_landmarks()
            if self.save_img:
                self._save_image()
                
        except Exception as e:
            print(f"Error processing image: {str(e)}")

    def _draw_eye_landmarks(self, landmarks, eye_landmarks, color):
        """
        Draw circles on the eye landmarks and display their indices.
        
        Args:
            landmarks: Detected face landmarks
            eye_landmarks: List of landmark indices for an eye
            color: Color to draw the landmarks
        """
        for idx in eye_landmarks:
            # Draw the circle for the landmark
            cv.circle(self.image, (landmarks[idx]), 6, color, cv.FILLED)
            
            # Add the landmark index as text above the circle
            text_pos = (landmarks[idx][0] - 10, landmarks[idx][1] - 10)  # Position text above circle
            cv.putText(self.image, str(idx), text_pos, 
                      cv.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    def plot_eye_landmarks(self):
        """Display the image with eye landmarks."""
        plt.figure(figsize=(10, 10))
        img_rgb = cv.cvtColor(self.image, cv.COLOR_BGR2RGB)
        plt.imshow(img_rgb)
        plt.axis('off')
        plt.show()
        
    def _save_image(self):
        """Save the processed image."""
        try:
            cv.imwrite(self.filename, self.image)
            print(f"Image saved successfully to {self.filename}")
        except Exception as e:
            print(f"Error saving image: {str(e)}")

# Example usage
if __name__ == "__main__":
    try:
        image_path = "DATA/IMAGES/DOWNLOADED_IMAGES/eyes_2.png"
        eye_landmarks = DetectEyeLandmarks(
            image_path, 
            save_img=True, filename="eye_landmarks_3.jpg"  
    )
    except Exception as e:
        print(f"Failed to process image: {str(e)}")
        

### Notes
1. **Right Eye vs. Left Eye**:
   - The code uses predefined indices for landmarks specific to the right and left eyes based on MediaPipe's FaceMesh model.

2. **Visualization**:
   - Green circles represent the right eye landmarks.
   - Red circles represent the left eye landmarks.
   - Landmark indices are displayed as text near each point.

3. **Error Handling**:
   - Handles errors for image loading and face detection, ensuring smooth execution.

4. **Next Step**:
   - After successfully detecting the eye landmarks, the next step is to compute the Eye Aspect Ratio (EAR). We fist highlighted the specific eye landmarks in both eyes which are taken into account for the calculation of EAR, after that we have calculated the EAR based on the formula mentioned in the next section


## 3. Drawing the specific EAR landmarks

The Eye Aspect Ratio (EAR) is a crucial metric in computer vision, used to measure the distance between key eye landmarks. It helps distinguish between regular blinks and prolonged eye closures, which can indicate drowsiness or fatigue. This metric is widely used in applications such as facial movement analysis, signal processing, and driver monitoring systems.

**Calculation**

The EAR is calculated by measuring the distances between the upper and lower eyelids and the corners of the eye. The formula, derived from the work by Soukupová and Čech in their 2016 paper, [*Real-Time Eye Blink Detection using Facial Landmarks*](https://vision.fe.uni-lj.si/cvww2016/proceedings/papers/05.pdf), is as follows:

$$ \text{EAR} = \frac{(p_2 - p_6) + (p_3 - p_5)}{2 \times (p_1 - p_4)} $$

where:

- $ p_1, p_2, p_3, p_4, p_5 $ and $p_6$ are the coordinates of 2D facial landmarks, such as the eyebrow, eyelid, and eye corner.
- The numerator calculates the vertical distance between the lower and upper eyelids.
- The denominator calculates the horizontal distance between the eye corners, weighted by 2, as there are two sets of horizontal eye landmarks but only one set of vertical eye landmarks.

The EAR remains relatively constant when the eyes are open but drops significantly when the eyes are closed, indicating a blink. However, a low EAR value doesn't always mean a blink. It could result from eye irritation, intentional eye closure, or facial expressions like smiling or yawning. Therefore, a larger temporal window is necessary to accurately declare a blink.

>The corresponding EAR points for left and right landmarks are
> - RIGHT_EAR : [33, 159, 158, 133, 153, 145]
> - LEFT_EAR : [362, 380, 374, 263, 386, 385]

The following code defines a class `DrawEARLandmarks` responsible for visualizing the specific eye landmarks involved in EAR calculation and displaying the EAR value at the top-left corner of the frame

In [ ]:
class DrawEARLandmarks:
    # Eye landmark indices
    LEFT_EYE_EAR = [362, 380, 374, 263, 386, 385]
    RIGHT_EYE_EAR = [33, 159, 158, 133, 153, 145]
    
    # Colors
    GREEN_COLOR = (0, 255, 0)  # Right eye color
    RED_COLOR = (0, 0, 255)   # Left eye color
    
    def __init__(self, image_path, save_img=False, filename=None):
        """
        Initialize the eye landmark detector.
        
        Args:
            image_path (str): Path to input image
            save_img (bool): Whether to save the processed image
            filename (str): Output filename if saving image
        """
        self.image_path = image_path
        self.filename = filename
        self.save_img = save_img
        self.generator = FaceMeshGenerator()
        
        # Setup save directory if needed
        if self.save_img:
            self._setup_save_directory()
            
        # Process image
        self.image = self._load_image()
        if self.image is not None:
            self._process_image()
    
    def _setup_save_directory(self):
        """Create directory for saving images if it doesn't exist."""
        if not self.filename:
            raise ValueError("Filename must be provided when save_img is True")
            
        save_dir = "DATA/IMAGES/BLINK_DETECTION"
        os.makedirs(save_dir, exist_ok=True)
        self.filename = os.path.join(save_dir, self.filename)

    def _load_image(self):
        """Load and validate input image."""
        try:
            img = cv.imread(self.image_path)
            if img is None:
                raise FileNotFoundError(f"Could not load image from {self.image_path}")
            return img
        except Exception as e:
            print(f"Error loading image: {str(e)}")
            return None

    def _eye_aspect_ratio(self, landmarks, eye_landmarks):
        """
        Calculate the Eye Aspect Ratio (EAR) for a given eye.
        
        Args:
            landmarks (list): Detected face landmarks
            eye_landmarks (list): List of landmark indices for an eye
        
        Returns:
            float: The calculated EAR value
        """
        A = np.linalg.norm(np.array(landmarks[eye_landmarks[1]]) - np.array(landmarks[eye_landmarks[5]]))
        B = np.linalg.norm(np.array(landmarks[eye_landmarks[2]]) - np.array(landmarks[eye_landmarks[4]]))
        C = np.linalg.norm(np.array(landmarks[eye_landmarks[0]]) - np.array(landmarks[eye_landmarks[3]]))
        ear = (A + B) / (2.0 * C)
        return ear
    
    def _draw_eye_landmarks(self, landmarks, eye_landmarks, color):
        """
        Draw circles on the eye landmarks and display their indices.
        
        Args:
            landmarks: Detected face landmarks
            eye_landmarks: List of landmark indices for an eye
            color: Color to draw the landmarks
        """
        for idx in eye_landmarks:
            # Draw the circle for the landmark
            cv.circle(self.image, (landmarks[idx]), 8, color, cv.FILLED)

        cv.line(self.image, landmarks[eye_landmarks[1]], landmarks[eye_landmarks[5]], (255, 255, 255), 4)
        cv.line(self.image, landmarks[eye_landmarks[2]], landmarks[eye_landmarks[4]], (255, 255, 255), 4)
        cv.line(self.image, landmarks[eye_landmarks[0]], landmarks[eye_landmarks[3]], (255, 255, 255), 4)

        left_ear = self._eye_aspect_ratio(landmarks, self.LEFT_EYE_EAR)
        right_ear = self._eye_aspect_ratio(landmarks, self.RIGHT_EYE_EAR)
        ear = (left_ear + right_ear) / 2.0

        DrawingUtils.draw_text_with_bg(self.image,  f"EAR : {ear:.2f}", (0, 80), font_scale=2, thickness=3)

    def _plot_eye_landmarks(self):
        """Display the image with eye landmarks."""
        plt.figure(figsize=(10, 10))
        img_rgb = cv.cvtColor(self.image, cv.COLOR_BGR2RGB)
        plt.imshow(img_rgb)
        plt.axis('off')
        plt.show()
        
    def _save_image(self):
        """Save the processed image."""
        try:
            cv.imwrite(self.filename, self.image)
            print(f"Image saved successfully to {self.filename}")
        except Exception as e:
            print(f"Error saving image: {str(e)}")

    def _process_image(self):
        """Process image to detect and draw landmarks."""
        try:
            # Detect face landmarks
            self.image, face_landmarks = self.generator.create_face_mesh(self.image, draw=False)
            
            if not face_landmarks:
                raise ValueError("No face detected in the image")
                
            # Draw landmarks for both eyes
            self._draw_eye_landmarks(face_landmarks, self.RIGHT_EYE_EAR, self.GREEN_COLOR)
            self._draw_eye_landmarks(face_landmarks, self.LEFT_EYE_EAR, self.RED_COLOR)
            
            # Display and save results
            self._plot_eye_landmarks()
            if self.save_img:
                self._save_image()
                
        except Exception as e:
            print(f"Error processing image: {str(e)}")

if __name__ == "__main__":
    image_path = "DATA/IMAGES/DOWNLOADED_IMAGES/eyes_3.png"
    DrawEARLandmarks(image_path, 
                    save_img=True, 
                    filename="eye_landmarks_3.jpg"  
                )


In [ ]:
DrawEARLandmarks("DATA/IMAGES/DOWNLOADED_IMAGES/eyes_1.jpg")

## 4. Analyzing the Eye Aspect Ratio(EAR) 

In the following code we processes a video to detect blinks by calculating the Eye Aspect Ratio (EAR) for both eyes in each frame. It uses a threshold-based approach to identify blinks and provides a real-time blink counter along with an EAR plot. 

Key Features:
1. **EAR Calculation**:
   - Computes EAR using vertical and horizontal eye distances.
   - EAR below a threshold indicates a possible blink.

2. **Blink Counter**:
   - Tracks consecutive frames with EAR below the threshold to count blinks.

3. **Visualization**:
   - Highlights detected eye landmarks in real-time.
   - Displays the EAR value and total blinks on the video.
   - Once the video processing is completed it will plot the EAR value as measured in each frame

4. **Output Options**:
   - Saves the processed video with blink counter annotations.
   - Generates a plot of EAR values over time.

Below is the implementation of the `PlotEAR` class, which handles video processing, blink detection, and visualization.


In [ ]:
class PlotEAR:
   
    def __init__(self, video_path, save_video=False, video_output=None, 
                 save_plot=False, plot_output=None):
        self.generator = FaceMeshGenerator() 
        self.video_path = video_path
        
        # Video saving parameters
        self.save_video = save_video
        self.video_output = video_output
        
        # Plot saving parameters
        self.save_plot = save_plot
        self.plot_output = plot_output
        
        # Eye landmarks
        self.RIGHT_EYE = [33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161, 246] 
        self.LEFT_EYE = [362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385, 384, 398]
        self.RIGHT_EYE_EAR = [33, 159, 158, 133, 153, 145]
        self.LEFT_EYE_EAR = [362, 380, 374, 263, 386, 385]
        
        # Blink detection parameters
        self.EAR_THRESHOLD = 0.275
        self.CONSEC_FRAMES = 4
        self.blink_counter = 0
        self.frame_counter = 0
        
        # Store EAR values for plotting
        self.ear_values = []
        self.frame_numbers = []
        
        # Colors (BGR format for OpenCV)
        self.GREEN_COLOR = (86, 241, 13)    # Bright green
        self.RED_COLOR = (30, 46, 209)      # Bright red

        # Define the output video file
        if self.save_video and self.video_output:
            save_dir = "DATA/VIDEOS/OUTPUTS"
            os.makedirs(save_dir, exist_ok=True)
            self.video_output = os.path.join(save_dir, self.video_output)

    def eye_aspect_ratio(self, eye_landmarks, landmarks):
        A = np.linalg.norm(np.array(landmarks[eye_landmarks[1]]) - np.array(landmarks[eye_landmarks[5]]))
        B = np.linalg.norm(np.array(landmarks[eye_landmarks[2]]) - np.array(landmarks[eye_landmarks[4]]))
        C = np.linalg.norm(np.array(landmarks[eye_landmarks[0]]) - np.array(landmarks[eye_landmarks[3]]))
        ear = (A + B) / (2.0 * C)
        return ear

    def set_colors(self, ear):
        return self.RED_COLOR if ear < self.EAR_THRESHOLD else self.GREEN_COLOR

    def draw_eye_landmarks(self, frame, landmarks, eye_landmarks, color):
        for loc in eye_landmarks:
            cv.circle(frame, (landmarks[loc]), 4, color, cv.FILLED)

    def plot_ear_values(self):
            
        # Set dark theme
        plt.style.use('dark_background')
        
        # Create figure and axis with dark background
        fig, ax = plt.subplots(figsize=(12, 6))
        fig.patch.set_facecolor('#000000')
        ax.set_facecolor('#000000')
        
        # Plot EAR values
        ax.plot(self.frame_numbers, self.ear_values, color='#00FF00', linewidth=1.5, 
                label=f'EAR (Total Blinks: {self.blink_counter})')
        
        # Add threshold line
        ax.axhline(y=self.EAR_THRESHOLD, color='#FF0000', linestyle='--', 
                  label=f'EAR_THRESHOLD : ({self.EAR_THRESHOLD})')
        
        # Customize grid
        ax.grid(True, linestyle='--', alpha=0.3)
        
        # Customize labels and title
        ax.set_xlabel('Frame Number', color='white', fontsize=12)
        ax.set_ylabel('Eye Aspect Ratio (EAR)', color='white', fontsize=12)
        ax.set_title('Eye Aspect Ratio Over Time', color='white', fontsize=18, pad=20, 
                     fontweight='bold')
        
        # Customize ticks
        ax.tick_params(colors='white')
        
        # Customize legend
        ax.legend(facecolor='#000000', edgecolor='white', labelcolor='white')
        
        # Adjust layout
        plt.tight_layout()
        
        # Determine plot filename
        if self.save_plot : 
            if self.plot_output :
                plot_save_dir = 'DATA/IMAGES/BLINK_DETECTION'
                os.makedirs(plot_save_dir, exist_ok=True)
                plot_filename = os.path.join(plot_save_dir, self.plot_output)
            else :
                # Generate default filename based on video path
                base_name = os.path.splitext(os.path.basename(self.video_path))[0]
                plot_filename = os.path.join(plot_save_dir, f"{base_name}_ear_plot.png")
            
            # Save plot
            fig.savefig(plot_filename, facecolor='#000000', edgecolor='none', bbox_inches='tight')
        plt.show()

    def process_video(self):
        try:
            cap = cv.VideoCapture(self.video_path)
            if not cap.isOpened():
                print(f"Failed to open video: {self.video_path}")
                raise IOError("Error: couldn't open the video!")

            w, h, fps = (int(cap.get(x)) for x in (
                cv.CAP_PROP_FRAME_WIDTH, 
                cv.CAP_PROP_FRAME_HEIGHT, 
                cv.CAP_PROP_FPS
            ))

            if self.save_video:
                self.out = cv.VideoWriter(
                    self.video_output, 
                    cv.VideoWriter_fourcc(*"mp4v"), 
                    fps, 
                    (w, h)
                )

            frame_num = 0
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break

                frame, face_landmarks = self.generator.create_face_mesh(frame, draw=False)

                if len(face_landmarks) > 0:
                    right_ear = self.eye_aspect_ratio(self.RIGHT_EYE_EAR, face_landmarks)
                    left_ear = self.eye_aspect_ratio(self.LEFT_EYE_EAR, face_landmarks)
                    ear = (right_ear + left_ear) / 2.0
                    
                    # Store EAR value and frame number
                    self.ear_values.append(ear)
                    self.frame_numbers.append(frame_num)

                    color = self.set_colors(ear)

                    if ear < self.EAR_THRESHOLD:
                        self.frame_counter += 1
                    else:
                        if self.frame_counter >= self.CONSEC_FRAMES:
                            self.blink_counter += 1
                        self.frame_counter = 0

                    self.draw_eye_landmarks(frame, face_landmarks, self.RIGHT_EYE, color)
                    self.draw_eye_landmarks(frame, face_landmarks, self.LEFT_EYE, color)

                    DrawingUtils.draw_text_with_bg(frame, f"Blinks: {self.blink_counter}", (0, 60), 
                                    font_scale=2, thickness=3,
                                    bg_color=color, text_color=(0, 0, 0))

                    if self.save_video:
                        self.out.write(frame)

                    resized_frame = cv.resize(frame, (1280, 720))
                    cv.imshow("Blink Counter", resized_frame)

                frame_num += 1
                if cv.waitKey(1) & 0xFF == ord('p'):
                    break

            cap.release()
            if self.save_video:
                self.out.release()
            cv.destroyAllWindows()
            
            # Generate plot after video processing
            if self.ear_values:
                self.plot_ear_values()

        except Exception as e:
            print(f"An error occurred: {e}")


# Example usage
if __name__ == "__main__":
    input_video_path = "DATA/VIDEOS/INPUTS/blinking_0.mp4"
    blink_counter = PlotEAR(
        video_path=input_video_path,
        # save_video=True,
        # video_output="blink_counter_4.mp4",
        save_plot=True,
        plot_output="blinking_0_ear_plot.png"
    )
    blink_counter.process_video()

### 4.1 Blink Detection with Multiple EAR Thresholds

This implementation extends blink detection by evaluating Eye Aspect Ratio (EAR) against multiple thresholds. By varying threshold values, this approach helps determine the optimal threshold or threshold range for accurate blink detection. 

#### Key Highlights:
1. **Threshold Variation**:
   - Multiple EAR thresholds are tested (e.g., `0.250`, `0.275`, `0.300`, and `0.325`).
   - Each threshold tracks its own blink count, enabling comparative analysis.

2. **Dynamic EAR Plotting**:
   - EAR values are plotted alongside threshold lines.
   - Each threshold line is color-coded for easy distinction.

3. **Output Options**:
   - Generates static and interactive plots to analyze EAR behavior with multiple thresholds.
   - Saves plots as `.png` and `.html` for future reference.

4. **Applications**:
   - Helps fine-tune the EAR threshold for specific use cases like drowsiness detection or fatigue monitoring.

Below is the implementation of the `BlinkCounterMultiThreshold` class, which processes the video, computes EAR, and evaluates blink detection across varying thresholds.

In [ ]:
class BlinkCounterMultiThreshold:
    """
    A class to detect and count eye blinks in a video using multiple EAR (Eye Aspect Ratio) thresholds.
    
    This class processes a video file, calculates the Eye Aspect Ratio for each frame,
    and generates a plot showing the EAR values over time with multiple threshold lines
    and an average EAR line.
    
    Attributes:
        video_path (str): Path to the input video file
        consec_frames (int): Number of consecutive frames below threshold to count as a blink
        EAR_thresholds (list): List of float values representing different EAR thresholds
        save_plot (bool): Whether to save the generated plot
        plot_output (str): Output filename for the saved plot
    """
    
    def __init__(self, video_path, consec_frames, EAR_thresholds, save_plot=False, plot_output=None):
        """
        Initialize the BlinkCounterMultiThreshold with video and analysis parameters.
        
        Args:
            video_path (str): Path to the input video file
            consec_frames (int): Number of consecutive frames below threshold to count as a blink
            EAR_thresholds (list): List of float values representing different EAR thresholds
            save_plot (bool, optional): Whether to save the generated plot. Defaults to False.
            plot_output (str, optional): Output filename for the saved plot. Defaults to None.
        """
        
        self.generator = FaceMeshGenerator() 
        self.video_path = video_path
        self.consec_frames = consec_frames
        self.EAR_thresholds = EAR_thresholds
       
        # Plot saving parameters
        self.save_plot = save_plot
        self.plot_output = plot_output
        
        # Eye landmarks
        self.RIGHT_EYE = [33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161, 246] 
        self.LEFT_EYE = [362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385, 384, 398]
        self.RIGHT_EYE_EAR = [33, 159, 158, 133, 153, 145]
        self.LEFT_EYE_EAR = [362, 380, 374, 263, 386, 385]
        
        # Store EAR values for plotting
        self.ear_values = []
        self.frame_numbers = []
       
    def eye_aspect_ratio(self, eye_landmarks, landmarks):
        """
        Calculate the Eye Aspect Ratio (EAR) for a given eye.
        
        The EAR is calculated using the formula:
        EAR = (||p2-p6|| + ||p3-p5||) / (2||p1-p4||)
        where p1, p2, p3, p4, p5, p6 are 2D landmark points.
        
        Args:
            eye_landmarks (list): List of indices for the eye landmarks
            landmarks (list): List of all facial landmarks coordinates
            
        Returns:
            float: The calculated Eye Aspect Ratio
        """
        A = np.linalg.norm(np.array(landmarks[eye_landmarks[1]]) - 
                           np.array(landmarks[eye_landmarks[5]]))
        B = np.linalg.norm(np.array(landmarks[eye_landmarks[2]]) - 
                           np.array(landmarks[eye_landmarks[4]]))
        C = np.linalg.norm(np.array(landmarks[eye_landmarks[0]]) - 
                           np.array(landmarks[eye_landmarks[3]]))
        ear = (A + B) / (2.0 * C)
        return ear

    def count_blinks(self, ear_threshold):
        """
        Count the number of blinks that occur below a given EAR threshold.
        
        A blink is counted when the EAR stays below the threshold for at least
        consec_frames number of frames.
        
        Args:
            ear_threshold (float): The EAR threshold below which to count blinks
            
        Returns:
            int: The total number of blinks detected
        """
        blink_count = 0
        frame_counter = 0
        
        for ear in self.ear_values:
            if ear < ear_threshold:
                frame_counter += 1
            else:
                if frame_counter >= self.consec_frames:
                    blink_count += 1
                frame_counter = 0
                
        return blink_count

    def plot_ear_values(self):
        """
        Generate an interactive plot of EAR values over time.
        
        Creates a plot showing:
        - EAR values over time
        - Average EAR line
        - Multiple threshold lines with blink counts
        - Dark theme with customized styling
        
        If save_plot is True, saves both static PNG and interactive HTML versions
        of the plot.
        """
        
        # Create figure with dark theme
        fig = go.Figure()
        
        # Calculate average EAR
        average_ear = sum(self.ear_values) / len(self.ear_values)
        
        # Plot EAR values
        fig.add_trace(go.Scatter(
            x=self.frame_numbers,
            y=self.ear_values,
            mode='lines',
            name='EAR',
            line=dict(color='#00FF00', width=2),
            hovertemplate='Frame: %{x}<br>EAR: %{y:.3f}<extra></extra>'
        ))
        
        # Add average EAR line
        fig.add_trace(go.Scatter(
            x=[min(self.frame_numbers), max(self.frame_numbers)],
            y=[average_ear, average_ear],
            mode='lines',
            name=f'Average EAR: {average_ear:.3f}',
            line=dict(color='#FFFFFF', width=2, dash='dot'),
        ))
        
        # Add threshold lines with different colors
        thresholds = self.EAR_thresholds
        colors = ['#FF0000', '#FF00FF', '#00FFFF', '#FFFF00']  # Red, Magenta, Cyan, Yellow
        
        for threshold, color in zip(thresholds, colors):
            blink_count = self.count_blinks(threshold)
            fig.add_trace(go.Scatter(
                x=[min(self.frame_numbers), max(self.frame_numbers)],
                y=[threshold, threshold],
                mode='lines',
                name=f'Threshold {threshold:.3f} (Blinks: {blink_count})',
                line=dict(color=color, width=2, dash='dash'),
            ))
        
        # Update layout with dark theme and borders
        fig.update_layout(
            template='plotly_dark',
            plot_bgcolor='black',
            paper_bgcolor='black',
            title=dict(
                text='Eye Aspect Ratio Over Time With Multiple Thresholds',
                font=dict(size=24, color='white'),
                x=0.5,
                y=0.95
            ),
            xaxis=dict(
                title='Frame Number',
                gridcolor='rgba(128, 128, 128, 0.2)',
                title_font=dict(size=16, color='white'),
                tickfont=dict(color='white'),
                showline=True,
                linewidth=1,
                linecolor='white',
                mirror=True  
            ),
            yaxis=dict(
                title='Eye Aspect Ratio (EAR)',
                gridcolor='rgba(128, 128, 128, 0.2)',
                title_font=dict(size=16, color='white'),
                tickfont=dict(color='white'),
                showline=True,
                linewidth=1,
                linecolor='white',
                mirror=True  
            ),
            showlegend=True,
            legend=dict(
                bgcolor='rgba(0,0,0,0)',
                bordercolor='rgba(255,255,255,0.2)',
                borderwidth=1,
                font=dict(color='white'),
                x=1.02,
                y=1
            ),
            hovermode='x unified',
            margin=dict(t=100, r=200),  # Adjust margins to prevent legend cutoff
            width=1400,  # Set specific width for consistent static image output
            height=700   # Set specific height for consistent static image output
        )
        
        # Determine plot filename and save if requested
        if self.save_plot:
            plot_save_dir = 'DATA/IMAGES/BLINK_DETECTION'
            os.makedirs(plot_save_dir, exist_ok=True)
            
            if self.plot_output:
                # Get the base filename without extension
                base_filename = os.path.splitext(self.plot_output)[0]
            else:
                base_filename = os.path.splitext(os.path.basename(self.video_path))[0]
                base_filename += "_multi_threshold_ear_plot"
            
            # Save as static image (PNG)
            png_filename = os.path.join(plot_save_dir, f"{base_filename}.png")
            fig.write_image(png_filename, scale=2)  # scale=2 for higher resolution
            
            # Save as HTML for interactive version
            html_filename = os.path.join(plot_save_dir, f"{base_filename}.html")
            fig.write_html(html_filename)
            
            print(f"Saved static plot as: {os.path.abspath(png_filename)}")
            print(f"Saved interactive plot as: {os.path.abspath(html_filename)}")
        
        # Show the interactive plot
        fig.show()

    def process_video(self):
        """
        Process the input video to detect eye blinks.
        
        This method:
        1. Opens the video file
        2. Processes each frame to detect facial landmarks
        3. Calculates EAR for both eyes in each frame
        4. Stores the average EAR values
        5. Generates a plot of the results
        
        Raises:
            IOError: If the video file cannot be opened
            Exception: For other errors during video processing
        """
        try:
            cap = cv.VideoCapture(self.video_path)
            if not cap.isOpened():
                print(f"Failed to open video: {self.video_path}")
                raise IOError("Error: couldn't open the video!")

            frame_num = 0
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break

                frame, face_landmarks = self.generator.create_face_mesh(frame, draw=False)

                if len(face_landmarks) > 0:
                    right_ear = self.eye_aspect_ratio(self.RIGHT_EYE_EAR, face_landmarks)
                    left_ear = self.eye_aspect_ratio(self.LEFT_EYE_EAR, face_landmarks)
                    ear = (right_ear + left_ear) / 2.0
                    
                    # Store EAR value and frame number
                    self.ear_values.append(ear)
                    self.frame_numbers.append(frame_num)

                frame_num += 1

            cap.release()
            cv.destroyAllWindows()
            
            # Generate plot after video processing
            if self.ear_values:
                self.plot_ear_values()

        except Exception as e:
            print(f"An error occurred: {e}")


# Example usage
if __name__ == "__main__":
    input_video_path = "DATA/VIDEOS/INPUTS/blinking_0.mp4"
    blink_counter = BlinkCounterMultiThreshold(
        video_path=input_video_path,
        consec_frames=4,
        EAR_thresholds=[0.250, 0.275, 0.300, 0.325],
        save_plot=True,
        plot_output="blinking_0_multi_threshold_ear_plot.png"
    )
    blink_counter.process_video()

#### Analysis
  From this analysis of different EAR thresholds with CONSEC_FRAMES set to 4, we can draw several important observations:

1. **Threshold Performance Analysis**:
  - *`0.250 (Red line)`: Only detected `8 blinks`*
    * This threshold is too low, missing many genuine blinks
    * Only captures the most extreme eye closures
    * Underestimates the actual blink count significantly

  - *` 0.275-0.300 (Purple and Blue lines)`: Both detected `23 blinks`*
    * This range appears to be optimal for blink detection
    * Consistently captures natural blink patterns
    * Shows the most accurate blink count
    * The agreement between these two thresholds suggests this is a stable detection range

  - *`0.325 (Yellow line)`: Detected `26 blinks`*
    * This threshold is too high
    * May be detecting false positives
    * Counting minor eye movements or partial blinks as full blinks
    * Overestimates the actual blink count

2. **Optimal Detection Range**:
  - The sweet spot appears to be between `0.275` and `0.300`
  - This range provides consistent and reliable blink detection
  - Matches typical EAR patterns during natural blinks
  - Offers good discrimination between blinks and normal eye movements

3. **Detection Reliability**:
  - The consistency between `0.275` and `0.300` thresholds (both detecting `23` blinks) suggests these are reliable settings
  - The divergence in counts at `0.250` and `0.325` indicates these are outside the optimal detection range

4. **Implementation Recommendations**:
  - Set the EAR threshold between `0.275` and `0.300` for optimal detection
  - Keep `CONSEC_FRAMES` at 4 as it works well with these thresholds
  - Avoid thresholds below `0.275` or above `0.300` to prevent missed detections or false positives

> This settings for `consec_frames` and `EAR_threshold` are strictly for the video `blinking_0.mp4` and may need to be adjusted for other videos.


Let's look for the optimal EAR_Threshold for this video `blinking_1.mp4`

In [ ]:
input_video_path = "DATA/VIDEOS/INPUTS/blinking_1.mp4"
blink_counter = BlinkCounterMultiThreshold(
    video_path=input_video_path,
    consec_frames=3, 
    EAR_thresholds=[0.265, 0.285, 0.295, 0.305],
    save_plot=True,
    plot_output="blinking_1_multi_threshold_ear_plot.png"
)
blink_counter.process_video()


### 4.2 Blink Detection with Varying Consecutive Frames

This implementation focuses on analyzing blink detection by varying the consecutive frame count (`consec_frames`) after determining an optimal Eye Aspect Ratio (EAR) threshold. This approach helps fine-tune the sensitivity of blink detection based on the required blink duration.

#### Key Highlights:
1. **Optimal Threshold Usage**:
   - The EAR threshold is fixed at a value (e.g., `0.285`), which was determined from prior analysis.

2. **Varying Frame Intervals**:
   - Different `consec_frames` values (e.g., `2`, `4`, and `6`) are used to evaluate their impact on blink detection accuracy.

3. **Plot with GridSpec Layout**:
   - The EAR timeline is visualized in a main plot with a fixed threshold line.
   - Subplots highlight specific frame intervals and blink counts for each `consec_frames` value.

4. **Applications**:
   - Helps determine the ideal `consec_frames` value for diverse scenarios (e.g., fast vs. slow blinking).
   - Enables dynamic adjustments to improve detection robustness in different environments.

Below is the implementation of the `EARConsecFrames` class, which processes the video, computes EAR values, and analyzes blink detection by varying `consec_frames`.


> The values of the blink detection parameters are chosen as per the video input `blinking_0.mp4`. 

In [ ]:
class EARConsecFrames:
    def __init__(self, video_path, Ear_threshold, consec_frames, save_plot=False, plot_output=None):
        self.generator = FaceMeshGenerator() 
        self.video_path = video_path
        
        # Plot saving parameters
        self.save_plot = save_plot
        self.plot_output = plot_output
        
        # Eye landmarks
        self.RIGHT_EYE = [33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161, 246] 
        self.LEFT_EYE = [362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385, 384, 398]
        self.RIGHT_EYE_EAR = [33, 159, 158, 133, 153, 145]
        self.LEFT_EYE_EAR = [362, 380, 374, 263, 386, 385]

        self.threshold = Ear_threshold
        self.consec_frames = consec_frames
        
        # Store EAR values for plotting
        self.ear_values = []
        self.frame_numbers = []
       
    def eye_aspect_ratio(self, eye_landmarks, landmarks):
        A = np.linalg.norm(np.array(landmarks[eye_landmarks[1]]) - np.array(landmarks[eye_landmarks[5]]))
        B = np.linalg.norm(np.array(landmarks[eye_landmarks[2]]) - np.array(landmarks[eye_landmarks[4]]))
        C = np.linalg.norm(np.array(landmarks[eye_landmarks[0]]) - np.array(landmarks[eye_landmarks[3]]))
        ear = (A + B) / (2.0 * C)
        return ear

    def count_blinks(self, ear_threshold, consec_frames):
        blink_count = 0
        frame_counter = 0
        
        for ear in self.ear_values:
            if ear < ear_threshold:
                frame_counter += 1
            else:
                if frame_counter >= consec_frames:
                    blink_count += 1
                frame_counter = 0
                
        return blink_count

    def plot_ear_values(self):
    # Set dark theme
        plt.style.use('dark_background')
        
        # Create figure with GridSpec
        fig = plt.figure(figsize=(16, 9))
        gs = fig.add_gridspec(2, 3)
        
        # Set global dark background
        fig.patch.set_facecolor('#000000')
        
        # Main plot spanning entire first row
        ax1 = fig.add_subplot(gs[0, :])
        ax1.set_facecolor('#000000')
        
        # Plot full EAR values
        ax1.plot(self.frame_numbers, self.ear_values, color='#00FF00', linewidth=1.5, label='EAR')
        
        # Fixed threshold
        threshold = self.threshold
        ax1.axhline(y=threshold, color='#FF00FF', linestyle='--', label=f'Threshold {threshold:.3f}')
        
        # Customize main plot
        ax1.grid(True, linestyle='--', alpha=0.5)
        ax1.set_xlabel('Frame Number', color='white', fontsize=12)
        ax1.set_ylabel('Eye Aspect Ratio (EAR)', color='white', fontsize=12)
        ax1.set_title('Complete Eye Aspect Ratio Timeline', 
                    color='white', fontsize=14, pad=10, fontweight='bold')
        ax1.tick_params(colors='white')
        ax1.legend(facecolor='#000000', edgecolor='white', labelcolor='white')
        
        # Create subplots for different intervals
        intervals = self.consec_frames
        colors = ['#FF0000', '#00FFFF', '#FFFF00']  # Red, Cyan, Yellow
        
        # Filter frame range 300-400
        start_idx, end_idx = 110, 170
        frames_subset = self.frame_numbers[start_idx:end_idx]
        ear_subset = self.ear_values[start_idx:end_idx]
        
        for idx, (interval, color) in enumerate(zip(intervals, colors)):
            # Create subplot in second row
            ax = fig.add_subplot(gs[1, idx])
            ax.set_facecolor('#000000')
            
            # Plot EAR values
            ax.plot(frames_subset, ear_subset, color='#00FF00', linewidth=2, label='EAR')
            ax.axhline(y=threshold, color='#FF00FF', linestyle='--', label=f'Threshold {threshold:.3f}')
            
            # Highlight regions based on interval
            for start in range(start_idx+interval, end_idx-interval, interval):
                ax.axvspan(start, start + interval, facecolor=color, alpha=0.1)
                ax.axvline(x=start, color=color, linestyle='--', alpha=0.8)
            
            # Count blinks in this range
            blink_count = self.count_blinks(threshold, interval)
            
            # Customize subplot
            ax.grid(True, linestyle='--', alpha=0.5)
            ax.set_xlabel('Frame Number', color='white', fontsize=10)
            ax.set_ylabel('EAR', color='white', fontsize=10)
            ax.set_title(f'Interval {interval} Frames\n{blink_count} blinks', 
                        color='white', fontsize=12, pad=10)
            ax.tick_params(colors='white')
            ax.set_xlim(start_idx, end_idx)
        
        # Adjust layout
        plt.tight_layout(h_pad=1.5)
        
        # Save plot if requested
        if self.save_plot:
            plot_save_dir = 'DATA/IMAGES/BLINK_DETECTION'
            os.makedirs(plot_save_dir, exist_ok=True)
            
            if self.plot_output:
                plot_filename = os.path.join(plot_save_dir, self.plot_output)
            else:
                base_name = os.path.splitext(os.path.basename(self.video_path))[0]
                plot_filename = os.path.join(plot_save_dir, f"{base_name}_gridspec_ear_plot.png")
            
            # Save plot
            fig.savefig(plot_filename, facecolor='#000000', edgecolor='none', bbox_inches='tight', dpi=300)
            
        plt.show()

    def process_video(self):
        try:
            cap = cv.VideoCapture(self.video_path)
            if not cap.isOpened():
                print(f"Failed to open video: {self.video_path}")
                raise IOError("Error: couldn't open the video!")

            frame_num = 0
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break

                frame, face_landmarks = self.generator.create_face_mesh(frame, draw=False)

                if len(face_landmarks) > 0:
                    right_ear = self.eye_aspect_ratio(self.RIGHT_EYE_EAR, face_landmarks)
                    left_ear = self.eye_aspect_ratio(self.LEFT_EYE_EAR, face_landmarks)
                    ear = (right_ear + left_ear) / 2.0
                    
                    # Store EAR value and frame number
                    self.ear_values.append(ear)
                    self.frame_numbers.append(frame_num)

                frame_num += 1

            cap.release()
            cv.destroyAllWindows()
            
            # Generate plot after video processing
            if self.ear_values:
                self.plot_ear_values()

        except Exception as e:
                print(f"An error occurred: {e}")


# Example usage
if __name__ == "__main__":
    input_video_path = "DATA/VIDEOS/INPUTS/blinking_0.mp4"
    blink_counter = EARConsecFrames(
        video_path=input_video_path,
        Ear_threshold=0.285,
        consec_frames=[2, 4, 6],
        # save_plot=True,
        # plot_output="lady_blinking_consec_frames_ear_plot_grid.png"
    )
    blink_counter.process_video()

In the code shell below we have used `Plolty` instead of `Matplotlib` for an interactive plot. 

In [ ]:
class EARConsecFramesPlotly:
    """
    A class for analyzing and visualizing Eye Aspect Ratio (EAR) from video data using Plotly.
    
    The class processes video frames to detect facial landmarks and calculate EAR values,
    then creates interactive visualizations showing EAR variations over time with different
    consecutive frame intervals for blink detection.
    
    Attributes:
        video_path (str): Path to the input video file
        threshold (float): EAR threshold value for blink detection
        consec_frames (list): List of consecutive frame intervals to analyze
        save_plot (bool): Whether to save the generated plots
        plot_output (str): Output filename for saved plots
    """
    def __init__(self, video_path, EAR_threshold, consec_frames, save_plot=False, plot_output=None):
        self.generator = FaceMeshGenerator() 
        self.video_path = video_path
        self.threshold = EAR_threshold
        self.consec_frames = consec_frames
        
        # Plot saving parameters
        self.save_plot = save_plot
        self.plot_output = plot_output
        
        # Eye landmarks
        self.RIGHT_EYE = [33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161, 246] 
        self.LEFT_EYE = [362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385, 384, 398]
        self.RIGHT_EYE_EAR = [33, 159, 158, 133, 153, 145]
        self.LEFT_EYE_EAR = [362, 380, 374, 263, 386, 385]
        
        # Store EAR values for plotting
        self.ear_values = []
        self.frame_numbers = []
       
    def eye_aspect_ratio(self, eye_landmarks, landmarks):
        """
        Calculate the Eye Aspect Ratio (EAR) for a given eye.
        
        The EAR is calculated using the formula:
        EAR = (||p2-p6|| + ||p3-p5||) / (2||p1-p4||)
        where p1, p2, p3, p4, p5, p6 are 2D landmark points.
        
        Args:
            eye_landmarks (list): List of indices for the eye landmarks
            landmarks (list): List of all facial landmarks coordinates
            
        Returns:
            float: The calculated Eye Aspect Ratio
        """
        A = np.linalg.norm(np.array(landmarks[eye_landmarks[1]]) - 
                           np.array(landmarks[eye_landmarks[5]]))
        B = np.linalg.norm(np.array(landmarks[eye_landmarks[2]]) - 
                           np.array(landmarks[eye_landmarks[4]]))
        C = np.linalg.norm(np.array(landmarks[eye_landmarks[0]]) - 
                           np.array(landmarks[eye_landmarks[3]]))
        ear = (A + B) / (2.0 * C)
        return ear

    def count_blinks(self, ear_threshold, consec_frames, start_idx=None, end_idx=None):
        """
        Count the number of blinks based on EAR values and consecutive frames criteria.
        
        Args:
            ear_threshold (float): Threshold value for considering an eye as closed
            consec_frames (int): Number of consecutive frames required for a blink
            start_idx (int, optional): Starting index for analysis
            end_idx (int, optional): Ending index for analysis
            
        Returns:
            int: Number of detected blinks
        """
        blink_count = 0
        frame_counter = 0

        if start_idx is not None and end_idx is not None:
            ear_values = self.ear_values[start_idx:end_idx]
        else:
            ear_values = self.ear_values
        
        for ear in ear_values:
            if ear < ear_threshold:
                frame_counter += 1
            else:
                if frame_counter >= consec_frames:
                    blink_count += 1
                frame_counter = 0
                
        return blink_count

    def plot_ear_values(self):
        """
        Generate interactive plots showing EAR values over time with various analysis metrics.
        
        Creates a multi-panel plot including:
        - Main plot showing complete EAR timeline
        - Three subplots showing different consecutive frame intervals
        - EAR threshold line
        - Average EAR line
        - Interval highlighting
        """
        # Create figure with secondary y-axis
        fig = make_subplots(
            rows=2, cols=3,
            specs=[
                [{'colspan':3}, None, None],
                [{}, {}, {}]
            ],
            subplot_titles=('Complete Eye Aspect Ratio Timeline', 
                        'Interval 2 Frames', 'Interval 4 Frames', 'Interval 6 Frames'),
            row_heights=[0.6, 0.4],
            vertical_spacing=0.2,
            horizontal_spacing=0.05
        )
        
        # Calculate average EAR
        avg_ear = np.mean(self.ear_values)
        
        # Fixed threshold
        threshold = self.threshold
        
        # Main plot (full timeline)
        fig.add_trace(
            go.Scatter(
                x=self.frame_numbers,
                y=self.ear_values,
                name='EAR',
                line=dict(color='#00FF00', width=1.5),
                hovertemplate='Frame: %{x}<br>EAR: %{y:.3f}<extra></extra>',
                legendgroup='ear',
                showlegend=True
            ),
            row=1, col=1
        )
        
        # Add threshold line to main plot
        fig.add_trace(
            go.Scatter(
                x=[min(self.frame_numbers), max(self.frame_numbers)],
                y=[threshold, threshold],
                name='EAR Threshold',
                line=dict(color='#FF00FF', width=1.5, dash='dash'),
                legendgroup='threshold',
                showlegend=True
            ),
            row=1, col=1
        )
        
        # Add average EAR line to main plot
        fig.add_trace(
            go.Scatter(
                x=[min(self.frame_numbers), max(self.frame_numbers)],
                y=[avg_ear, avg_ear],
                name='Average EAR',
                line=dict(color='#FFFFFF', width=1.5, dash='dot'),
                legendgroup='average',
                showlegend=True
            ),
            row=1, col=1
        )
        
        # Filter frame range for subplots
        start_idx, end_idx = 400, 500
        frames_subset = self.frame_numbers[start_idx:end_idx]
        ear_subset = self.ear_values[start_idx:end_idx]
        
        # Colors for interval highlighting
        colors = ['#FF0000', '#00FFFF', '#FFFF00']  # Red, Cyan, Yellow
        intervals = self.consec_frames
        
        # Create subplots for different intervals
        for idx, (interval, color) in enumerate(zip(intervals, colors)):
            # Add EAR trace
            fig.add_trace(
                go.Scatter(
                    x=frames_subset,
                    y=ear_subset,
                    name=f'EAR (interval {interval})',
                    line=dict(color='#00FF00', width=2),
                    hovertemplate='Frame: %{x}<br>EAR: %{y:.3f}<extra></extra>',
                    legendgroup='ear',
                    showlegend=False,
                ),
                row=2, col=idx+1
            )
            
            # Add threshold line
            fig.add_trace(
                go.Scatter(
                    x=[min(frames_subset), max(frames_subset)],
                    y=[threshold, threshold],
                    name=f'EAR Threshold (interval {interval})',
                    line=dict(color='#FF00FF', width=1.5, dash='dash'),
                    legendgroup='threshold',
                    showlegend=False,
                ),
                row=2, col=idx+1
            )
            
            # Add average EAR line
            fig.add_trace(
                go.Scatter(
                    x=[min(frames_subset), max(frames_subset)],
                    y=[avg_ear, avg_ear],
                    name=f'Average EAR (interval {interval})',
                    line=dict(color='#FFFFFF', width=1.5, dash='dot'),
                    legendgroup='average',
                    showlegend=False,
                ),
                row=2, col=idx+1
            )
            
            # Add interval highlighting
            for start in range(start_idx+interval, end_idx-interval, interval):
                fig.add_vrect(
                    x0=start,
                    x1=start + interval,
                    fillcolor=color,
                    opacity=0.1,
                    line_width=0,
                    row=2,
                    col=idx+1
                )
                
                fig.add_vline(
                    x=start,
                    line_color=color,
                    line_dash="3px",
                    line_width=1,
                    opacity=0.8,
                    row=2,
                    col=idx+1
                )
            
            # Count blinks for both total timeline and subset
            total_blinks = self.count_blinks(threshold, interval)
            subset_blinks = self.count_blinks(threshold, interval, start_idx, end_idx)
            
            # Update subplot title to include both blink counts
            fig.layout.annotations[idx+1].update(
                text=f'Interval {interval} Frames<br>Total: {total_blinks} blinks<br>Subset: {subset_blinks} blinks'
            )
        
        # Update layout
        fig.update_layout(
            template="plotly_dark",
            paper_bgcolor='black',
            plot_bgcolor='black',
            title=dict(
                text=f"Eye Aspect Ratio Analysis",
                font=dict(size=30),
                y=0.97,
                x=0.5,
                xanchor='center',
                yanchor='top'
            ),
            showlegend=True,
            legend=dict(
                bgcolor='rgba(0,0,0,0)',
                bordercolor='rgba(255,255,255,0.2)',
                borderwidth=1,
                font=dict(color='white'),
                orientation='h',
                xanchor='right',
                yanchor='bottom',
                x=1,
                y=1.02,
                groupclick="togglegroup"
            ),
            hovermode='x unified',
            height=900,
            width=1600,
        )
        
        for i in range(1, 5):  # Loop through all subplots (1 main + 3 small)
            if i == 1:
                row, col = 1, 1
                xaxis_limit = [min(self.frame_numbers), max(self.frame_numbers)]
            else:
                row, col = 2, i-1
                xaxis_limit = [min(frames_subset), max(frames_subset)]
                
            # Update x-axis
            fig.update_xaxes(
                showline=True,
                range=xaxis_limit, 
                linewidth=1,
                linecolor='white',
                mirror=True,
                title='Frame Number',
                row=row,
                col=col
            )
            
            # Update y-axis
            fig.update_yaxes(
                showline=True,
                linewidth=1,
                linecolor='white',
                mirror=True,
                title='Eye Aspect Ratio (EAR)' if col == 1 else '',
                row=row,
                col=col
            )
        
        # Save plot if requested
        if self.save_plot:
            plot_save_dir = 'DATA/IMAGES/BLINK_DETECTION'
            os.makedirs(plot_save_dir, exist_ok=True)
            
            if self.plot_output:
                base_filename = os.path.splitext(self.plot_output)[0]
            else:
                base_filename = os.path.splitext(os.path.basename(self.video_path))[0]
                base_filename += "_consec_frames_ear_plot_plotly"
            
            png_filename = os.path.join(plot_save_dir, f"{base_filename}.png")
            fig.write_image(png_filename, scale=2)
            
            html_filename = os.path.join(plot_save_dir, f"{base_filename}.html")
            fig.write_html(html_filename)
            
            print(f"Saved static plot as: {os.path.abspath(png_filename)}")
            print(f"Saved interactive plot as: {os.path.abspath(html_filename)}")
        
        fig.show()

    def process_video(self):
        """
        Process the input video to extract EAR values and generate visualization.
        
        Reads the video frame by frame, detects facial landmarks, calculates EAR values,
        and stores them for later analysis and visualization.
        
        Raises:
            IOError: If the video file cannot be opened
            Exception: For other processing errors
        """
        try:
            cap = cv.VideoCapture(self.video_path)
            if not cap.isOpened():
                print(f"Failed to open video: {self.video_path}")
                raise IOError("Error: couldn't open the video!")

            frame_num = 0
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break

                frame, face_landmarks = self.generator.create_face_mesh(frame, draw=False)

                if len(face_landmarks) > 0:
                    right_ear = self.eye_aspect_ratio(self.RIGHT_EYE_EAR, face_landmarks)
                    left_ear = self.eye_aspect_ratio(self.LEFT_EYE_EAR, face_landmarks)
                    ear = (right_ear + left_ear) / 2.0
                    
                    self.ear_values.append(ear)
                    self.frame_numbers.append(frame_num)

                frame_num += 1

            cap.release()
            cv.destroyAllWindows()
            
            if self.ear_values:
                self.plot_ear_values()

        except Exception as e:
                print(f"An error occurred: {e}")


# Example usage
if __name__ == "__main__":
    input_video_path = "DATA/VIDEOS/INPUTS/blinking_0.mp4"
    blink_counter = EARConsecFramesPlotly(
        video_path=input_video_path,
        EAR_threshold=0.285,
        consec_frames=[2, 4, 6],
        save_plot=True,
        plot_output="blinking_0_consec_frames_ear_plot_grid_plotly.png"
    )
    blink_counter.process_video()

#### Observations from the plots:
We initially set the EAR_threshold to 0.28 and varied the number of consecutive_frames to find out the optimal value for `CONSEC_FRAMES`
1. **Blink Detection Across Frame Intervals:**
   - For `consec_frames = 2`: 
     - The system detects **23 blinks**, which matches the total expected number of blinks. 
     - This shows high accuracy as the smaller interval captures the rapid changes in EAR associated with blinks.
   - For `consec_frames = 4`:
     - The system again detects **23 blinks**, demonstrating that this interval is still effective for blink detection without missing any events.
   - For `consec_frames = 6`:
     - The system detects **10 blinks**, missing many blinks compared to the expected total (23). 
     - This suggests that larger intervals result in skipped frames where EAR dips, leading to under-detection.

2. **Subset Analysis:**
   - In the selected subset of frames (around 400–500), the number of detected blinks decreases as the frame interval increases.
     - For `consec_frames = 2` and `4`, the subset shows **3 blinks**—indicating accurate tracking.
     - For `consec_frames = 6`, the subset shows only **1 blink**, confirming that some blink events are being overlooked.

3. **Trade-off Between Frame Interval and Blink Detection:**
   - Smaller intervals ensure better detection accuracy as they capture transient EAR changes during blinks.
   - Larger intervals fail to record quick events, as they skip over frames where EAR dips below the threshold.

When using 6 consecutive frames (`consec_frames = 6`), some blinks are missed because the system does not sample frames frequently enough to capture the transient changes in the Eye Aspect Ratio (EAR) during a blink. Here's a detailed explanation:

##### 1. **Nature of Blinks and EAR Changes**
   - A blink is a rapid action where the EAR drops sharply and returns to normal within a short time (typically a few frames in a high-frame-rate video).
   - If the interval between sampled frames is too large (e.g., 6 frames), the system may "skip over" the frames where the EAR dips below the threshold (indicating a blink). As a result:
     - The EAR might never appear below the threshold within the sampled frames.
     - The blink goes undetected.


##### 2. **Temporal Resolution and Sampling Frequency**
   - Smaller `consec_frames` (e.g., 2 or 4) provide higher temporal resolution, meaning the system samples the EAR values more frequently, ensuring that even rapid changes in EAR are captured.
   - Larger `consec_frames` reduce the sampling frequency, causing the system to lose fine-grained information about quick events like blinks.



##### 3. **Comparison of Frame Intervals**
   - Consider an EAR timeline where a blink occurs between frames 10 and 12:
     - **For `consec_frames = 2` or `4`**: The system samples EAR values at frames 10, 12, 14, etc., ensuring it captures the dip at frame 10 or 12.
     - **For `consec_frames = 6`**: The system samples EAR values at frames 6, 12, 18, etc. In this case:
       - If the blink starts and ends between sampled frames (e.g., between frames 8 and 11), the system completely misses it.



##### 4. **Impact of Blink Duration**
   - A blink typically lasts only a few milliseconds (around 100–400 ms for humans). 
   - If the frame interval exceeds the duration of the blink (as it might when `consec_frames = 6`), the system skips the blink entirely.



##### Visual Summary
Imagine this as a simplified timeline for EAR changes:

| Frame Number | EAR Value | Blink Detected? (Threshold = 0.28) |
|--------------|-----------|-----------------------------------|
| 10           | 0.35      | No                               |
| 11           | 0.25      | Yes (Blink starts)               |
| 12           | 0.22      | Yes (Blink continues)            |
| 13           | 0.27      | Yes (Blink ends)                 |
| 14           | 0.32      | No                               |

- **`consec_frames = 2`:** Captures frames 10, 12, 14 → Blink detected.
- **`consec_frames = 6`:** Captures frames 10, 16 → Blink missed, as the system skips critical frames where the EAR drops.


##### Key Takeaway:
A larger frame interval (e.g., `6`) reduces the likelihood of detecting short-duration events like blinks because the system skips frames where the EAR temporarily drops below the threshold. To avoid this, it's crucial to select an interval that ensures the system can capture the rapid changes in EAR during a blink.


We observed from the EAR graph that to count the blinks properly we need to keep the `EYE_THRESHOLD` in the range 0.275 to 0.3 and the `CONSEC_FRAMES` in the range 2-4 for the video input `blinking_0.mp4` as it is providing correct and stable value of total number of blinks.

## 5. Blink Detection and Counting

This code provides a robust implementation for detecting and counting blinks in a video by leveraging the Eye Aspect Ratio (EAR). After determining the optimal EAR threshold and consecutive frame count (`consec_frames`), this script accurately detects blinks in real-time or from recorded videos.

### Key Features:
1. **EAR-Based Blink Detection**:
   - Uses facial landmarks around the eyes to calculate the EAR.
   - Blinks are detected when the EAR drops below the threshold (`ear_threshold`) for a specified number of consecutive frames.

2. **Customizable Parameters**:
   - `ear_threshold`: Defines the EAR value below which eyes are considered closed.
   - `consec_frames`: Determines how long the eyes must stay closed to count as a blink.

3. **Visualization**:
   - Landmarks around the eyes are visualized in green (open) or red (closed) based on the EAR value.
   - A live counter displays the total number of blinks detected.

4. **Video Processing**:
   - Processes each frame to detect blinks and can optionally save the processed video with overlays.


In [ ]:
class BlinkCounter:
    """
    A class to detect and count eye blinks in a video using facial landmarks.
    
    This class processes video input to detect eye blinks by calculating the Eye Aspect Ratio (EAR)
    of both eyes. It can either process a video file or save the processed output to a new video file.
    
    Attributes:
        ear_threshold (float): Threshold below which eyes are considered closed
        consec_frames (int): Number of consecutive frames eyes must be closed to count as blink
    """
    
    def __init__(self, video_path, ear_threshold, consec_frames, save_video=False, output_filename=None):
        """
        Initialize the BlinkCounter with video source and processing parameters.
        
        Args:
            video_path (str): Path to the input video file
            ear_threshold (float): Threshold for eye aspect ratio to detect blink (default: 0.3)
            consec_frames (int): Number of consecutive frames needed to confirm blink (default: 4)
            save_video (bool): Whether to save the processed video
            output_filename (str, optional): Name for the output video file
        """
        # Initialize face mesh detector
        self.generator = FaceMeshGenerator() 
        self.video_path = video_path
        self.save_video = save_video
        self.output_filename = output_filename
        
        # Define facial landmarks for eye detection
        # Each list contains indices corresponding to points around the eyes
        self.RIGHT_EYE = [33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161, 246]
        self.LEFT_EYE = [362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385, 384, 398]
        
        # Specific landmarks for EAR calculation
        # These specific points are used to calculate the eye aspect ratio
        self.RIGHT_EYE_EAR = [33, 159, 158, 133, 153, 145]
        self.LEFT_EYE_EAR = [362, 380, 374, 263, 386, 385]
        
        # Blink detection parameters
        self.ear_threshold = ear_threshold  # Eye aspect ratio threshold for blink detection
        self.consec_frames = consec_frames  # Minimum consecutive frames for a valid blink
        self.blink_counter = 0    # Counter for total blinks detected
        self.frame_counter = 0    # Counter for consecutive frames below threshold
        
        # Define colors for visualization (in BGR format)
        self.GREEN_COLOR = (86, 241, 13)  # Used when eyes are open
        self.RED_COLOR = (30, 46, 209)    # Used when eyes are closed
        
        # Set up output video directory and path if saving is enabled
        if self.save_video and self.output_filename:
            save_dir = "DATA/VIDEOS/OUTPUTS"
            os.makedirs(save_dir, exist_ok=True)
            self.output_filename = os.path.join(save_dir, self.output_filename)

    def update_blink_count(self, ear):
        """
        Update blink counter based on current eye aspect ratio.
        
        This method implements the blink detection logic:
        - If EAR is below threshold, increment frame counter
        - If EAR returns above threshold and enough consecutive frames were counted,
          increment blink counter
        
        Args:
            ear (float): Current eye aspect ratio
            
        Returns:
            bool: True if a new blink was detected, False otherwise
        """
        blink_detected = False
        
        if ear < self.ear_threshold:
            self.frame_counter += 1
        else:
            if self.frame_counter >= self.consec_frames:
                self.blink_counter += 1
                blink_detected = True
            self.frame_counter = 0
            
        return blink_detected

    def eye_aspect_ratio(self, eye_landmarks, landmarks):
        """
        Calculate the eye aspect ratio (EAR) for given eye landmarks.
        
        The EAR is calculated using the formula:
        EAR = (||p2-p6|| + ||p3-p5||) / (2||p1-p4||)
        where p1-p6 are specific points around the eye.
        
        Args:
            eye_landmarks (list): Indices of landmarks for one eye
            landmarks (list): List of all facial landmarks
        
        Returns:
            float: Calculated eye aspect ratio
        """
        A = np.linalg.norm(np.array(landmarks[eye_landmarks[1]]) - np.array(landmarks[eye_landmarks[5]]))
        B = np.linalg.norm(np.array(landmarks[eye_landmarks[2]]) - np.array(landmarks[eye_landmarks[4]]))
        C = np.linalg.norm(np.array(landmarks[eye_landmarks[0]]) - np.array(landmarks[eye_landmarks[3]]))
        return (A + B) / (2.0 * C)

    def set_colors(self, ear):
        """
        Determine visualization color based on eye aspect ratio.
        
        Args:
            ear (float): Current eye aspect ratio
        
        Returns:
            tuple: BGR color values
        """
        return self.RED_COLOR if ear < self.ear_threshold else self.GREEN_COLOR

    def draw_eye_landmarks(self, frame, landmarks, eye_landmarks, color):
        """
        Draw landmarks around the eyes on the frame.
        
        Args:
            frame (numpy.ndarray): Video frame to draw on
            landmarks (list): List of facial landmarks
            eye_landmarks (list): Indices of landmarks for one eye
            color (tuple): BGR color values for drawing
        """
        for loc in eye_landmarks:
            cv.circle(frame, (landmarks[loc]), 4, color, cv.FILLED)

    def process_video(self):
        """
        Main method to process the video and detect blinks.
        
        This method:
        1. Opens the video file
        2. Processes each frame to detect faces and calculate EAR
        3. Counts blinks based on EAR values
        4. Displays and optionally saves the processed video
        
        Raises:
            IOError: If video file cannot be opened
            Exception: For other processing errors
        """
        try:
            # Open video capture
            cap = cv.VideoCapture(self.video_path)
            if not cap.isOpened():
                print(f"Failed to open video: {self.video_path}")
                raise IOError("Error: couldn't open the video!")

            # Get video properties
            w, h, fps = (int(cap.get(x)) for x in (
                cv.CAP_PROP_FRAME_WIDTH,
                cv.CAP_PROP_FRAME_HEIGHT,
                cv.CAP_PROP_FPS
            ))

            # Initialize video writer if saving is enabled
            if self.save_video:
                self.out = cv.VideoWriter(
                    self.output_filename,
                    cv.VideoWriter_fourcc(*"mp4v"),
                    fps,
                    (w, h)
                )

            # Main processing loop
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break

                # Detect facial landmarks
                frame, face_landmarks = self.generator.create_face_mesh(frame, draw=False)

                if len(face_landmarks) > 0:
                    # Calculate eye aspect ratio
                    right_ear = self.eye_aspect_ratio(self.RIGHT_EYE_EAR, face_landmarks)
                    left_ear = self.eye_aspect_ratio(self.LEFT_EYE_EAR, face_landmarks)
                    ear = (right_ear + left_ear) / 2.0

                    # Update blink detection
                    self.update_blink_count(ear)

                    # Determine visualization color based on EAR
                    color = self.set_colors(ear)

                    # Draw visualizations
                    self.draw_eye_landmarks(frame, face_landmarks, self.RIGHT_EYE, color)
                    self.draw_eye_landmarks(frame, face_landmarks, self.LEFT_EYE, color)
                    DrawingUtils.draw_text_with_bg(frame, f"Blinks: {self.blink_counter}", (0, 60),
                                    font_scale=2, thickness=3,
                                    bg_color=color, text_color=(0, 0, 0))

                    # Save frame if enabled
                    if self.save_video:
                        self.out.write(frame)

                    # Display the frame
                    resized_frame = cv.resize(frame, (1280, 720))
                    cv.imshow("Blink Counter", resized_frame)

                # Break loop if 'p' is pressed
                if cv.waitKey(int(1000/fps)) & 0xFF == ord('p'):
                    break

            # Cleanup
            cap.release()
            if self.save_video:
                self.out.release()
            cv.destroyAllWindows()

        except Exception as e:
            print(f"An error occurred: {e}")


# Example usage
if __name__ == "__main__":
    input_video_path = "DATA/VIDEOS/INPUTS/blinking_4.mp4"
    
    # Create blink counter with custom parameters
    blink_counter = BlinkCounter(
        video_path=input_video_path,
        ear_threshold=0.3,  
        consec_frames=4,    
        # save_video=True,
        # output_filename="blink_counter_4.mp4"
    )
    blink_counter.process_video()

## 6. Real-Time Eye Blink Detection and Visualization with EAR Plotting

This Python script implements a **real-time blink detection system** using the Eye Aspect Ratio (EAR). The system processes video frames to detect eye blinks, count them, and visualize the EAR over time using a live plot. Additionally, the output video can be saved with the EAR plot overlay for further analysis.

---

### **Features**
1. **Blink Detection**  
   - Uses the Eye Aspect Ratio (EAR) method to detect blinks.
   - Counts blinks when EAR falls below a specified threshold for consecutive frames.

2. **Real-Time Visualization**  
   - Plots the EAR in real-time.
   - Displays EAR threshold and dynamically updates the plot based on frame data.

3. **Video Processing**  
   - Processes input video frames.
   - Option to save the processed video with EAR visualization.

4. **Customization**  
   - EAR threshold and consecutive frames for blink detection can be configured.
   - Adjustable plot aesthetics for enhanced readability.

---

### **Core Components**

#### **1. Eye Aspect Ratio (EAR) Calculation**
The EAR is calculated using specific points around the eyes to monitor the openness of the eyes:
$$
EAR = \frac{||p_2 - p_6|| + ||p_3 - p_5||}{2 \cdot ||p_1 - p_4||}
$$
Where $p_1, p_2, \dots, p_6$ are the eye landmarks.

#### **2. Real-Time Plot**
- The EAR plot updates live during video processing.
- Visual indicators like a threshold line make it easier to track blinks.

#### **3. Blink Counting Logic**
- If EAR falls below the threshold for a predefined number of frames (`consec_frames`), a blink is registered.

#### **4. Modular Design**
The class provides functions for:
   - Frame-by-frame processing.
   - EAR calculation.
   - Visualization of EAR values on a live plot.
   - Saving the output video with visualized data.

---

### **Usage Example**

#### **Inputs**
- **Video File**: The video to be processed.
- **EAR Threshold**: The value below which a blink is detected (e.g., 0.294).
- **Consecutive Frames**: The minimum number of consecutive frames below the threshold to count as a blink (e.g., 3).

#### **Outputs**
- Blink count displayed on the video.
- Real-time EAR plot overlay.
- Optionally saves the processed video with visualizations.

#### **Sample Code**
```python
if __name__ == "__main__":
    # Define input video path and parameters
    input_video_path = "DATA/VIDEOS/INPUTS/blinking_1.mp4"
    
    # Initialize the blink counter with parameters
    blink_counter = BlinkCounterandEARPlot(
        video_path=input_video_path,
        threshold=0.294,  # EAR threshold
        consec_frames=3,  # Frames below threshold to count as a blink
        save_video=True,  # Save processed video
        output_filename="blinking_1_output.mp4"  # Output filename
    )
    
    # Process the video and count blinks
    blink_counter.process_video()
```

---

### **Class Breakdown**

#### **1. `BlinkCounterandEARPlot`**
Main class encapsulating all functionality.

- **Attributes**:
  - `EAR_THRESHOLD`: Blink detection threshold.
  - `CONSEC_FRAMES`: Frames required to detect a blink.
  - `video_path`: Path to the input video.
  - `save_video`: Boolean flag to save the processed video.
  - `output_filename`: Filename for the saved video.

- **Methods**:
  - `_init_video_saving`: Configures video saving.
  - `_init_plot`: Initializes the live EAR plot.
  - `eye_aspect_ratio`: Calculates EAR for eye landmarks.
  - `process_frame`: Processes a single frame to calculate EAR and detect blinks.
  - `process_video`: Processes the entire video file frame-by-frame.
  - `_update_plot`: Updates the EAR plot with new values.


---

### **Expected Output**
- **Processed Video**:  
   A video with blink counts and a real-time EAR plot overlay.
- **Real-Time Visualization**:  
   EAR plot updates dynamically while processing the video.

---

This system is ideal for applications in **health monitoring**, **fatigue detection**, and **behavior analysis**, making it a versatile tool for real-time blink analysis.

In [ ]:

class BlinkCounterandEARPlot:
    """
    A class to detect and count eye blinks in a video using facial landmarks.
    
    This class processes video frames to detect faces, track eye movements,
    calculate Eye Aspect Ratio (EAR), plot EAR, and count blinks in real-time.
    """
    
    # Define facial landmark indices for eyes
    RIGHT_EYE = [33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161, 246]
    LEFT_EYE = [362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385, 384, 398]
    RIGHT_EYE_EAR = [33, 159, 158, 133, 153, 145]  # Points for EAR calculation
    LEFT_EYE_EAR = [362, 380, 374, 263, 386, 385]  # Points for EAR calculation
    
    # Define colors for visualization
    COLORS = {
        'GREEN': {'hex': '#56f10d', 'bgr': (86, 241, 13)},
        'BLUE': {'hex': '#0329fc', 'bgr': (30, 46, 209)},
        'RED': {'hex': '#f70202', 'bgr': None}
    }

    def __init__(self, video_path, threshold, consec_frames, save_video=False, output_filename=None):
        """
        Initialize the BlinkCounter with video and detection parameters.
        
        Args:
            video_path (str): Path to the input video file
            threshold (float): EAR threshold for blink detection
            consec_frames (int): Number of consecutive frames below threshold to count as a blink
            save_video (bool): Whether to save the processed video
            output_filename (str): Name of the output video file if saving
        """
        # Initialize core parameters
        self.generator = FaceMeshGenerator()
        self.video_path = video_path
        self.EAR_THRESHOLD = threshold
        self.CONSEC_FRAMES = consec_frames
        
        # Initialize video saving parameters
        self._init_video_saving(save_video, output_filename)
        
        # Initialize tracking variables
        self._init_tracking_variables()
        
        # Initialize plotting
        self._init_plot()

    def _init_video_saving(self, save_video, output_filename):
        """Initialize video saving parameters and create output directory if needed."""
        self.save_video = save_video
        self.output_filename = output_filename
        self.out = None
        
        if self.save_video and self.output_filename:
            save_dir = "DATA/VIDEOS/OUTPUTS"
            os.makedirs(save_dir, exist_ok=True)
            self.output_filename = os.path.join(save_dir, self.output_filename)

    def _init_tracking_variables(self):
        """Initialize variables used for tracking blinks and frame processing."""
        self.blink_counter = 0
        self.frame_counter = 0
        self.frame_number = 0
        self.ear_values = []
        self.frame_numbers = []
        self.max_frames = 100
        self.new_w = self.new_h = None
        self.default_ymin = 0.18  
        self.default_ymax = 0.44  

    def _init_plot(self):
        """Initialize the matplotlib plot for EAR visualization."""
        # Set up dark theme plot
        plt.style.use('dark_background')
        plt.ioff()
        self.fig, self.ax = plt.subplots(figsize=(8, 5), dpi=200)
        self.canvas = FigureCanvas(self.fig)
        
        # Configure plot aesthetics
        self._configure_plot_aesthetics()
        
        # Initialize plot data
        self._init_plot_data()

        self.fig.canvas.draw()

    def _configure_plot_aesthetics(self):
        """Configure the aesthetic properties of the plot."""
        # Set background colors
        self.fig.patch.set_facecolor('#000000')
        self.ax.set_facecolor('#000000')
        
        # Configure axes with default limits initially
        self.ax.set_ylim(self.default_ymin, self.default_ymax)
        self.ax.set_xlim(0, self.max_frames)
        
        # Set labels and title
        self.ax.set_xlabel("Frame Number", color='white', fontsize=12)
        self.ax.set_ylabel("EAR", color='white', fontsize=12)
        self.ax.set_title("Real-Time Eye Aspect Ratio (EAR)", 
                         color='white', pad=10, fontsize=18, fontweight='bold')
        
        # Configure grid and spines
        self.ax.grid(True, color='#707b7c', linestyle='--', alpha=0.7)
        for spine in self.ax.spines.values():
            spine.set_color('white')
        
        # Configure ticks and legend
        self.ax.tick_params(colors='white', which='both')

    def _init_plot_data(self):
        """Initialize the plot data and curves."""
        self.x_vals = list(range(self.max_frames))
        self.y_vals = [0] * self.max_frames
        self.Y_vals = [self.EAR_THRESHOLD] * self.max_frames
        
        # Create curves with explicit labels
        self.EAR_curve, = self.ax.plot(
            self.x_vals, 
            self.y_vals,
            color=self.COLORS['GREEN']['hex'],
            label="Eye Aspect Ratio",
            linewidth=2
        )
        
        self.threshold_line, = self.ax.plot(
            self.x_vals,
            self.Y_vals,
            color=self.COLORS['RED']['hex'],
            label="Blink Threshold",
            linewidth=2,
            linestyle='--'
        )
        
        # Add legend 
        self.legend = self.ax.legend(
            handles=[self.EAR_curve, self.threshold_line],
            loc='upper right',
            fontsize=10,
            facecolor='black',
            edgecolor='white',
            labelcolor='white',
            framealpha=0.8,
            borderpad=1,
            handlelength=2
        )

    def eye_aspect_ratio(self, eye_landmarks, landmarks):
        """
        Calculate the eye aspect ratio (EAR) for given eye landmarks.
        
        The EAR is calculated using the formula:
        EAR = (||p2-p6|| + ||p3-p5||) / (2||p1-p4||)
        where p1-p6 are specific points around the eye.
        
        Args:
            eye_landmarks (list): Indices of landmarks for one eye
            landmarks (list): List of all facial landmarks
        
        Returns:
            float: Calculated eye aspect ratio
        """
        A = np.linalg.norm(np.array(landmarks[eye_landmarks[1]]) - 
                          np.array(landmarks[eye_landmarks[5]]))
        B = np.linalg.norm(np.array(landmarks[eye_landmarks[2]]) - 
                          np.array(landmarks[eye_landmarks[4]]))
        C = np.linalg.norm(np.array(landmarks[eye_landmarks[0]]) - 
                          np.array(landmarks[eye_landmarks[3]]))
        return (A + B) / (2.0 * C)

    def _update_plot(self, ear):
        """Update the plot with new EAR values."""
        if len(self.ear_values) > self.max_frames:
            self.ear_values.pop(0)
            self.frame_numbers.pop(0)
            
        color = self.COLORS['BLUE']['hex'] if ear < self.EAR_THRESHOLD else self.COLORS['GREEN']['hex']
        
        self.EAR_curve.set_xdata(self.frame_numbers)
        self.EAR_curve.set_ydata(self.ear_values)
        self.EAR_curve.set_color(color)
        
        self.threshold_line.set_xdata(self.frame_numbers)
        self.threshold_line.set_ydata([self.EAR_THRESHOLD] * len(self.frame_numbers))
        
        
        if len(self.frame_numbers) > 1:
            x_min = min(self.frame_numbers)
            x_max = max(self.frame_numbers)
            if x_min == x_max:
                # Add a small padding if min and max are the same
                x_min -= 0.5
                x_max += 0.5
            self.ax.set_xlim(x_min, x_max)
        else:
            # Default limits for initialization
            self.ax.set_xlim(0, self.max_frames)

        # Ensure the legend remains visible
        if self.legend not in self.ax.get_children():
            self.legend = self.ax.legend(
                handles=[self.EAR_curve, self.threshold_line],
                loc='upper right',
                fontsize=10,
                facecolor='black',
                edgecolor='white',
                labelcolor='white',
                framealpha=0.8,
                borderpad=1,
                handlelength=2
            )
        
        # Redraw with better quality
        self.ax.draw_artist(self.ax.patch)
        self.ax.draw_artist(self.EAR_curve)
        self.ax.draw_artist(self.threshold_line)
        self.ax.draw_artist(self.legend)
        self.fig.canvas.flush_events()

    def process_frame(self, frame):
        """
        Process a single frame to detect and analyze eyes.
        
        Returns:
            tuple: Processed frame and EAR value
        """
        frame, face_landmarks = self.generator.create_face_mesh(frame, draw=False)
        
        if not face_landmarks:
            return frame, None
            
        # Calculate EAR
        right_ear = self.eye_aspect_ratio(self.RIGHT_EYE_EAR, face_landmarks)
        left_ear = self.eye_aspect_ratio(self.LEFT_EYE_EAR, face_landmarks)
        ear = (right_ear + left_ear) / 2.0
        
        # Determine visualization color
        color = self.COLORS['BLUE']['bgr'] if ear < self.EAR_THRESHOLD else self.COLORS['GREEN']['bgr']
        
        # Draw landmarks and update blink counter
        self._draw_frame_elements(frame, face_landmarks, color)
        
        return frame, ear

    def _draw_frame_elements(self, frame, landmarks, color):
        """Draw eye landmarks and blink counter on the frame."""
        # Draw eye landmarks
        for eye in [self.RIGHT_EYE, self.LEFT_EYE]:
            for loc in eye:
                cv.circle(frame, (landmarks[loc]), 2, color, cv.FILLED)
        
        # Draw blink counter
        DrawingUtils.draw_text_with_bg(
            frame, f"Blinks: {self.blink_counter}", (0, 60),
            font_scale=2, thickness=3,
            bg_color=color, text_color=(0, 0, 0)
        )

    def process_video(self):
        """Process the entire video and detect blinks."""
        try:
            cap = cv.VideoCapture(self.video_path)
            if not cap.isOpened():
                raise IOError(f"Failed to open video: {self.video_path}")

            self._process_video_frames(cap)
            
        except Exception as e:
            print(f"An error occurred: {e}")
        finally:
            cap.release()
            if self.out:
                self.out.release()
            cv.destroyAllWindows()

    def _process_video_frames(self, cap):
        """Process individual frames from the video capture."""
        # Get video properties
        w = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
        h = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))
        fps = int(cap.get(cv.CAP_PROP_FPS))

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Process frame and get EAR
            frame, ear = self.process_frame(frame)
            
            if ear is not None:
                self._update_blink_detection(ear)
                self._update_visualization(frame, ear, fps)

            if cv.waitKey(1) & 0xFF == ord('p'):
                break

    def _update_blink_detection(self, ear):
        """Update blink detection based on EAR value."""
        self.ear_values.append(ear)
        self.frame_numbers.append(self.frame_number)
        
        if ear < self.EAR_THRESHOLD:
            self.frame_counter += 1
        else:
            if self.frame_counter >= self.CONSEC_FRAMES:
                self.blink_counter += 1
            self.frame_counter = 0
        
        self.frame_number += 1

    def _update_visualization(self, frame, ear, fps):
        """Update the visualization including the plot and video output."""
        self._update_plot(ear)
        
        # Convert plot to image and resize
        plot_img = self.plot_to_image()
        plot_img_resized = cv.resize(
            plot_img,
            (frame.shape[1], int(plot_img.shape[0] * frame.shape[1] / plot_img.shape[1]))
        )
        
        # Stack frames and handle video output
        stacked_frame = cv.vconcat([frame, plot_img_resized])
        self._handle_video_output(stacked_frame, fps)

    def _handle_video_output(self, stacked_frame, fps):
        """Handle video output, including saving and display."""
        # Initialize video writer if needed
        if self.new_w is None:
            self.new_w = stacked_frame.shape[1]
            self.new_h = stacked_frame.shape[0]
            if self.save_video:
                self.out = cv.VideoWriter(
                    self.output_filename,
                    cv.VideoWriter_fourcc(*"mp4v"),
                    fps,
                    (self.new_w, self.new_h)
                )

        # Save frame if requested
        if self.save_video:
            self.out.write(stacked_frame)

        # Display frame
        resizing_factor = 0.4
        resized_shape = (
            int(resizing_factor * stacked_frame.shape[1]),
            int(resizing_factor * stacked_frame.shape[0])
        )
        stacked_frame_resized = cv.resize(stacked_frame, resized_shape)
        cv.imshow("Video with EAR Plot", stacked_frame_resized)

    def plot_to_image(self):
        """Convert the matplotlib plot to an OpenCV-compatible image."""
        self.canvas.draw()
        
        buffer = self.canvas.buffer_rgba()
        img_array = np.asarray(buffer)
        
        # Convert RGBA to RGB
        img_rgb = cv.cvtColor(img_array, cv.COLOR_RGBA2RGB)
        return img_rgb


if __name__ == "__main__":
    # Example usage
    input_video_path = "DATA/VIDEOS/INPUTS/blinking_1.mp4"
    blink_counter = BlinkCounterandEARPlot(
        video_path=input_video_path,
        threshold=0.294,
        consec_frames=3,
        save_video=True,
        output_filename="blinking_1_output.mp4"
    )
    blink_counter.process_video()

In [ ]:
input_video_path = "DATA/VIDEOS/INPUTS/blinking_0.mp4"
blink_counter = BlinkCounter(
    video_path=input_video_path,
    threshold=0.275,
    consec_frames=4,
    save_video=True,
    output_filename="blinking_0_output.mp4"
)
blink_counter.process_video()

## 7. Applications

1. **Driver Safety Systems**: Real-time monitoring of driver blink patterns to detect drowsiness and fatigue, triggering alerts or emergency protocols to prevent accidents. These systems are increasingly being integrated into modern vehicles and fleet management solutions.

2. **Medical Diagnostics**: 
   - Detection of neurological conditions through abnormal blink patterns
   - Assessment of facial nerve function after surgery or injury
   - Monitoring of patients with conditions like [Bell's palsy](https://www.ninds.nih.gov/health-information/disorders/bells-palsy#:~:text=Bell's%20palsy%20is%20a%20neurological,injured%20or%20stops%20working%20properly.)
   - Early detection of conditions like [Parkinson's disease](https://pubmed.ncbi.nlm.nih.gov/2265915/#:~:text=Parkinson's%20disease%20is%20associated%20with,striatal%20and%20mesolimbic%20dopaminergic%20activity.) which can affect blink rates

3. **Human-Computer Interaction**:
   - Assistive technology for people with mobility impairments, using blinks as input signals
   - Virtual reality systems that track blinks to reduce eye strain
   - Gaming interfaces that incorporate blink detection for more immersive experiences

4. **Security and Authentication**:
   - Liveness detection in facial recognition systems to prevent spoofing
   - Biometric authentication using unique blink patterns
   - Monitoring systems for high-security environments

5. **Healthcare Monitoring**:
   - Remote patient monitoring systems
   - Sleep studies and sleep disorder diagnosis
   - Assessment of medication effects on neurological function

## 8. References
- [MediaPipe Face Mesh](https://ai.google.dev/edge/mediapipe/solutions/vision/face_landmarker)
- [Eye Aspect Ratio (EAR)](https://medium.com/analytics-vidhya/eye-aspect-ratio-ear-and-drowsiness-detector-using-dlib-a0b2c292d706)
- [Dewi, C., Chen, R., Chang, C., Wu, S., Jiang, X., & Yu, H. (2021). Eye Aspect Ratio for Real-Time Drowsiness Detection to Improve Driver Safety. Electronics, 11(19), 3183](https://doi.org/10.3390/electronics11193183).
- [Soukupová, E., & Čech, M. (2016). Real-Time Eye Blink Detection using Facial Landmarks. *Computer Vision and Applications*, 2016(1), 1-10](https://doi.org/10.1007/s00521-016-0934-z).

